# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，其符号为阳刚之卦，代表天、云、雷等现象。乾卦是由两个阴爻夹一个阳爻构成，象征天、云、雷等现象。它反映了阳刚之气、创造力和领导能力。乾卦的意义包括：创造、领导、坚定、进展、积极等。在八卦图中，乾卦位于北方，与事业、努力、刚健、果敢等有关。


In [8]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是八卦之一，也是八宫图之一，其符号为阴柔之卦，代表地、水、云等现象。地水师卦是由两个阴爻夹一个阳爻构成，象征地、水、云等现象。它反映了阴柔之美、灵活性和适应能力。地水师卦的意义包括：适应、变化、流动、沟通、合作等。在八卦图中，地水师卦位于西南方，与情感、家庭、沟通、合作等有关。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-handmade"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [11]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [12]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的一个卦象，由六个阳爻（表示刚强、积极、正义）和一条阴爻（表示柔顺、和谐、宁静）组成。乾卦象征着天、阳气、强健等含义，因此名为“乾”。

乾卦的卦辞为：“乾，元亨利贞。。”意指乾卦具有原始、亨通、正道的特性，是至高无上的卦象。

在《易经》的六十四卦中，乾卦是第一卦，也是最重要的一卦。它反映了宇宙的运行规律、人生的道德修养以及社会的发展原则。乾卦教导人们要刚强、积极、正义，要遵循自然法则和社会伦理，才能达到和谐、平衡、成功的境界。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？
乾卦是八卦之一，也是八宫图说中的“北方玄武七宫”。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、云彩等，也象征着刚强、积极、进取、勇敢、坚定等。在八宫图说中，乾卦位于北方，与事业、努力、决断、果敢、积极等有关，与健康、婚姻、家庭等无关。在易经中，乾卦代表阳刚之气，也象征着宇宙的创造力和发展力。乾卦的意义不仅限于八卦，还包含了人们对于宇宙、自然、人生等方面的理解和探索。


In [13]:
base_response, ft_response = compare_chatglm_results(query="地水师卦")

问题：地水师卦

原始输出：
"地水师卦" (Qiáojiāo shì shuǐ) is a Chinese phrase that can be translated as "spirit water fish". It is a term used in the I Ching, an ancient divination text, to describe a hexagram that represents a favorable set of circumstances.

The "spirit water fish" hexagram is the 37th hexagram in the I Ching and is associated with the element water and the卦卦辞 ("卦辞") describe it as follows:

"In the lower trigram is water,
In the upper trigram is a fish.
When the fish jumps out of the water,
It swims and becomes great."

This hexagram symbolizes the power of water to flow and adapt, and the importance of being flexible and adaptable in order to succeed. It encourages the seeker to be open-minded and responsive to changing circumstances, just as the fish in the hexagram is able to adapt and grow in different environments.

The "spirit water fish" hexagram is also associated with the idea of spiritual growth and self-improvement, and it reminds the seeker to remain humble and open to 

In [14]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
讼卦（卦象：乾上，坤下）卦辞：天与地相交， clouds gather and散，雷电交加，百川归海。象辞：的天象征刚强，地的险阻，刚强相遇，必有一争。象义：诉讼卦象征诉讼或争端，只有以柔克刚，以低矮的个子去战胜高大的个子，才能获得胜利。退而求其次，以小的去战胜大的，也是获得胜利的一种办法。这表明，在处理争端时，应采取退让、温柔的态度，以柔克刚。


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦 讼卦原文：讼。有攸往，得朋友。无朋友，无攸往。象曰：天与水相交，刚柔相济。君子以朋友终日三交，安其位而其正。白话文解释：讼卦象征诉讼。卦象由两个阴爻夹一个阳爻构成，表示刚柔之力的相互交融。君子观此卦象，取法于天与水的相交，刚柔相济，以刚柔之力的交融来达到和谐。启示君子要与朋友保持刚柔之力的相互交融，共谋大事。得此卦者，建议君子以朋友终日三交，安其位而其正，意为君子应安于自己的地位，坚守正道。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [16]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B (Epoch=3, automade-dataset) 微调后：\n{ft_response}")
    return base_response, ft_response

In [17]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。其卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。"

在传统解卦中，乾卦预示着大吉大利，事业如日中天，但也提醒要警惕盛极必衰的道理。经商方面顺利发展，但要冷静分析形势，坚持商业道德。对于婚恋，尽管阳盛阴衰，但刚柔可相济，最终形成美满结果。

在解卦时，乾卦表示充满活力和力量，预示着前方有希望，前方有路，充满阳光和乐观。应该抓住机遇，积极行动，一往无前，实现目标。在事业、经商、求名、婚恋等方面，都应充满信心，但同时也应保持谦逊谨慎的态度，积极应对各种挑战，最终达成目标。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。其卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。"


卦象：乾卦由六个阳爻组成，排列成纯阳之象，象征着刚健强劲的特性。

卦辞：乾卦的卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。

运势：乾卦预示着大吉大利，事业如日中天，但也提醒要警惕盛极必衰的道理。

事业：在事业中，乾卦表示刚健强劲的特性，预示着成功。

婚恋：婚恋中，乾卦象征着刚健强劲的特性，预示着重逢旧情。

财运：财运中，乾卦预示着富贵不禁，努力即可得到回报。

时机：在时机上，乾卦预示着万事刚脆，需要观时待机，时机成熟则事业成功。

结论：乾卦代表着刚健强劲的特性，预示着吉利如意，同时也提示要警惕盛极必衰的道理。在事业、婚恋、财运和时机等方面，乾卦都预示着成功和财富，但需要保持谦逊谨慎的态度，才能持续稳定地取得成功。


In [18]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，地水师卦象诚心敬德，得水地之间，仁者安之。


据南宋易学家朱熹解，地水师卦象水之盛也，得地之平，养军国之众，位高权重，军旅安乐。


据明末清初的易学家章太炎解，地水师卦意味着战事不利，兵旅听天由命，兵强则食，军旅不安，军旅离散，上下相争。


综合以上解卦，地水师卦具有丰富的哲学内涵，意味着战事不利，需要靠养兵国、训练兵旅、得人和众的支撑，才能支撑不久。因此，兵旅不安，军旅离散，上下相争，难以成就大事。在决策过程中，应该避免冒险，保持谨慎，稳定军心，以期获得胜利。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，地水师卦象者，主言仁义，仁义者，仁者见者亦仁，敬者尚敬人。


据南宋易学家朱熹解，地水师卦，言兵，军中水火相克，兵权未定，宜谨慎从事。


据明末清初的易学家陈希夷解，地水师卦，象征险阻，困难重重，必须谨慎小心，不可冒险，以确保安全。


综合来看，地水师卦预示着困难重重的境遇，必须谨慎小心，以保证安全。同时也提醒着人们，在困难的情况下，要坚守正道，以确保成功。


In [19]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
{'name': '讼卦"上卦为乾(天)，下卦为坎(水)，两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。', 'content': '\n讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。\n\n讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。\n\n对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。\n\n讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。'}


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 周易中的讼卦是什么卦象

 在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [22]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240217_113548"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [23]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [24]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [25]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
在周易中，地水师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地水相融的情景，寓意着君子应当像地水一样兼容并蓄，团结一致，才能成就大业。

地水师卦的特点是：吉祥无灾， predicted in the future. The main meaning is that if you can accommodate and make friendly with the earth, water and fire elements, you will be successful in your pursuits.

地水师卦的解释：君子观此卦象，应当以地载水，地载火，火生土，木生金，金生水，水生木，水又载火，形成一个环环相扣的完整循环。君子应当兼容并蓄，模擬大自然中的和谐，从而得到成功。

地水师卦的代表人物是坤卦（地）和坎卦（水）的代表，坤卦代表顺从和适应，坎卦代表吉祥无灾。在事业、经商、求名、婚恋等方面的运势中，地水师卦都预示着吉祥兼容、和谐共生。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 在周易中，地水师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地水相济的情景，寓意着君子应当像地一样容纳容纳大象。君子观此卦象，应当以容纳和等待为主，等待时机，避免冒险，从而获得吉祥。地水师卦象辞中提到：君子观此卦象，应当容纳此卦，象征地接受此卦。像地一样容纳大象，像水一样流动，融合与此卦象，自然吉祥。然而，此卦象也预示着君子需要像地一样容纳容纳，等待时机，像水一样适应环境，灵活应对。

地水师卦是一个吉祥的卦象，但需要等待时机，避免冒险。在事业、经商、求名、婚恋等方面的决策中，都需要谨慎行事，等待最佳时机，而非盲目行动。地水师卦的核心哲学是，等待时机，灵活应对，融合与此卦象，自然吉祥。


In [26]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。
